In [1]:
# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np
# from juqcs import Juqcs
# 假设有 3 台机器和 3 个操作
m = 3 # 机器数量
n = 2  # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3],[3, 1, 2]])

# 常数
K1 = 30
P = 30

# 量子比特数量
num_qubits = n * m + l

In [2]:
#
def pauli_z_term(index, num_qubits):
    """初始化一个 Pauli-I 门的列表,指定索引 index 替换为 Pauli-Z 门"""
    paulis = ['I'] * num_qubits
    paulis[index] = 'Z'
    return ''.join(paulis)  # 将列表转化为字符串

def pauli_x_term(index, num_qubits):
    paulis = ['I'] * num_qubits
    paulis[index] = 'X'
    return ''.join(paulis)
def pauli_y_term(index, num_qubits):
    paulis = ['I'] * num_qubits
    paulis[index] = 'Y'
    return ''.join(paulis)

In [3]:
from qiskit.quantum_info import Pauli, SparsePauliOp, PauliList

# 常数项
# constant_term = 1/2 * sum(2**h for h in range(l))
pauli_strings = []
cofficients = []
# 构建 Pauli-Z 项
z_terms = []
for h in range(l):
    coef = - 2**h / 2
    z_term = pauli_z_term(m * n + h, num_qubits)
    z_terms.append((coef, z_term))

# 从z_terms中提取Pauli字符串和系数
pauli_strings = [term[1] for term in z_terms]
coefficients = [term[0] for term in z_terms]
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian1 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))
print(hamiltonian1)

SparsePauliOp(['IIIIIIZ'],
              coeffs=[-0.5+0.j])


In [4]:
pauli_strings = []
cofficients = []
# 常数项
# constant_term1 = K1 * n * ((m / 2) - 1)**2
# pauli_strings.append('I'*num_qubits)
# cofficients.append(constant_term1)

# 线性项
x_terms = []
# coef_linear = -1 * K1 * (m / 2 - 1)
for i in range(n):
    for j in range(m):
        coef = -1 * K1 * (m - 1)/2
        x_term = pauli_z_term(i * m + j, num_qubits)
        pauli_strings.append(x_term)
        cofficients.append(coef)
        # linear_terms.append((coef, x_term))

# 二次项
# quadratic_terms = []
# coef_quadratic = K1 / 4
for i in range(n):
    coef = K1 / 4
    for j1 in range(m):
        for j2 in range(m):
            if j1 != j2:
                x_term1 = pauli_z_term(i * m + j1, num_qubits)
                x_term2 = pauli_z_term(i * m + j2, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term1, x_term2)])
                pauli_strings.append(combined_op)
                cofficients.append(coef)
            # quadratic_terms.append((coef, x_term1 + x_term2)) # 二次项的具体形式应该是每对 Zij 和 Zik 的组合

# 组合所有项
# pauli_strings = [term[1] for term in linear_terms + quadratic_terms]
# coefficients = [term[0] for term in linear_terms + quadratic_terms]
print(pauli_strings)
print(cofficients)
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian2 = SparsePauliOp.from_list(list(zip(pauli_strings, cofficients)))

print(hamiltonian2)

['ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI']
[-30.0, -30.0, -30.0, -30.0, -30.0, -30.0, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5]
SparsePauliOp(['ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI'],
              coeffs=[-30. +0.j, -30. +0.j, -30. +0.j, -30. +0.j, -30. +0.j, -30. +0.j,
   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,
   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j])


In [5]:
# 初始化数组
pauli_strings = []
coefficients = []
x_terms = []
z_terms = []
# 线性和二次项
for j in range(m):
    # constant_term = -1/2*(np.sum(p[:, j]) - ( 2 ** l-1))
    # h_terms = []

    for i in range(n):
        coef =-1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * p[i, j] * P
        x_term = pauli_z_term(i * m + j, num_qubits)
        x_terms.append((coef, x_term))
        # pauli_strings.append(x_term)
        # cofficients.append(coef)

    for h in range(l):
        coef = -1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * - (2 ** h) * P
        z_term = pauli_z_term(m * n + h, num_qubits)
        # pauli_strings.append(z_term)
        # cofficients.append(coef)
        x_terms.append((coef,z_term))

    for i1 in range(n):
        for i2 in range(n):
            if i1 != i2:
                coef = 1/4 * p[i1, j] * p[i2, j] * P
                x_term1 = pauli_z_term(i1 * m + j, num_qubits)
                x_term2 = pauli_z_term(i2 * m + j, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term1, x_term2)])
                # pauli_strings.append(combined_op)
                # cofficients.append(coef)
                x_terms.append((coef, combined_op))
    for i in range(n):
        for h in range(l):
            coef = 1/4 * -2 * p[i,j] * 2 ** h * P
            x_term = pauli_z_term(i * m + j, num_qubits)
            z_term = pauli_z_term(m * n + h, num_qubits)
            combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term, z_term)])
            # pauli_strings.append(combined_op)
            # cofficients.append(coef)
            x_terms.append((coef, combined_op))
    for h1 in range(l):
        for h2 in range(l):
            if h1 != h2:
                coef = 1/4 * 2 ** h1 * 2 ** h2 * P
                z_term1 = pauli_z_term(m * n + h1, num_qubits)
                z_term2= pauli_z_term(m * n + h2, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(z_term1, z_term2)])
                # pauli_strings.append(combined_op)
                # cofficients.append(coef)
                x_terms.append((coef, combined_op))
pauli_strings = [term[1] for term in x_terms]
coefficients = [term[0] for term in x_terms]
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian3 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(hamiltonian3)

SparsePauliOp(['ZIIIIII', 'IIIZIII', 'IIIIIIZ', 'ZIIZIII', 'ZIIZIII', 'ZIIIIIZ', 'IIIZIIZ', 'IZIIIII', 'IIIIZII', 'IIIIIIZ', 'IZIIZII', 'IZIIZII', 'IZIIIIZ', 'IIIIZIZ', 'IIZIIII', 'IIIIIZI', 'IIIIIIZ', 'IIZIIZI', 'IIZIIZI', 'IIZIIIZ', 'IIIIIZZ'],
              coeffs=[ -45. +0.j, -135. +0.j,   45. +0.j,   22.5+0.j,   22.5+0.j,  -15. +0.j,
  -45. +0.j,  -60. +0.j,  -30. +0.j,   30. +0.j,   15. +0.j,   15. +0.j,
  -30. +0.j,  -15. +0.j, -180. +0.j, -120. +0.j,   60. +0.j,   45. +0.j,
   45. +0.j,  -45. +0.j,  -30. +0.j])


In [6]:
cost_hamiltonian = hamiltonian1+hamiltonian2+hamiltonian3
print(cost_hamiltonian)

SparsePauliOp(['IIIIIIZ', 'ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI', 'ZIIIIII', 'IIIZIII', 'IIIIIIZ', 'ZIIZIII', 'ZIIZIII', 'ZIIIIIZ', 'IIIZIIZ', 'IZIIIII', 'IIIIZII', 'IIIIIIZ', 'IZIIZII', 'IZIIZII', 'IZIIIIZ', 'IIIIZIZ', 'IIZIIII', 'IIIIIZI', 'IIIIIIZ', 'IIZIIZI', 'IIZIIZI', 'IIZIIIZ', 'IIIIIZZ'],
              coeffs=[  -0.5+0.j,  -30. +0.j,  -30. +0.j,  -30. +0.j,  -30. +0.j,  -30. +0.j,
  -30. +0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,
    7.5+0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,    7.5+0.j,
    7.5+0.j,  -45. +0.j, -135. +0.j,   45. +0.j,   22.5+0.j,   22.5+0.j,
  -15. +0.j,  -45. +0.j,  -60. +0.j,  -30. +0.j,   30. +0.j,   15. +0.j,
   15. +0.j,  -30. +0.j,  -15. +0.j, -180. +0.j, -120. +0.j,   60. +0.j,
   45. +0.j,   45. +0.j,  -45. +0.j,  -30. +0.j])


In [7]:
# 初始化数组
pauli_strings = []
coefficients = []
x_terms = []
z_terms = []
for i in range(num_qubits):
    coef = 1
    x_term = pauli_x_term(i,num_qubits)
    x_terms.append((coef, x_term))
# # 设置前 n*m 个量子比特为 XY 混合器
# xy_terms = []
# for i in range(n):
#     for j in range(m):
#         # 确定量子比特对 (i * m + j) 和 (i * m + j + 1)
#         if j < m - 1:
#             coef = 0.5
#
#             # 生成Pauli-X和Pauli-Y操作的字符串
#             x_term = 'I' * (i * m + j) + 'X' + 'I' * (num_qubits - i * m - j - 1)
#             y_term = 'I' * (i * m + j) + 'Y' + 'I' * (num_qubits - i * m - j - 1)
#             x_term_next = 'I' * (i * m + j + 1) + 'X' + 'I' * (num_qubits - i * m - j - 2)
#             y_term_next = 'I' * (i * m + j + 1) + 'Y' + 'I' * (num_qubits - i * m - j - 2)
#
#             # 组合成 XY 项
#             xy_term_x = x_term[:i * m + j] + 'X' + x_term_next[i * m + j + 1:]
#             xy_term_y = y_term[:i * m + j] + 'Y' + y_term_next[i * m + j + 1:]
#
#             # 将生成的XY项添加到xy_terms中
#             xy_terms.append((coef, xy_term_x))
#             xy_terms.append((coef, xy_term_y))

# 提取所有的 Pauli 字符串和系数
pauli_strings = [term[1] for term in  x_terms ]
coefficients = [term[0] for term in  x_terms ]

# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
mixer_hamiltonian = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(mixer_hamiltonian)

SparsePauliOp(['XIIIIII', 'IXIIIII', 'IIXIIII', 'IIIXIII', 'IIIIXII', 'IIIIIXI', 'IIIIIIX'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


In [8]:
def invert_counts(s):
    return s[::-1]

In [9]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import Aer, transpile, assemble, QuantumCircuit, execute


# 定义成本函数
def cost_function(params, n, m, l, K1, P, p):
    beta, gamma = params[:len(params)//2], params[len(params)//2:]
    num_qubits = n * m + l
    qc = QuantumCircuit(num_qubits, num_qubits)  # 添加经典寄存器

    # 初始化量子态到 |+> 态
    qc.h(range(n * m + l))

    for g, b in zip(gamma, beta):
        # 应用问题哈密顿量
        qc.append(PauliEvolutionGate(cost_hamiltonian, g), list(range(num_qubits)))
        # 应用混合哈密顿量
        qc.append(PauliEvolutionGate(mixer_hamiltonian, b), list(range(num_qubits)))

    # print(qc)
    # 模拟电路
    # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)
    # 计算期望值
    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[(n*m):]])
        x_ij = np.array([int(bit) for bit in bitstring[:(n*m)]]).reshape(n, m)

        E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
        E2 = K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)])
        # alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
        E3 = 0
        for j in range(m):

            sum_xp = np.sum([x_ij[i][j] * p[i][j] for i in range(n)])

            sum_zh = np.sum([2**h * z_h[h] for h in range(l)])

            max_value = max(0, sum_xp - sum_zh)

            E3 += P * (max_value)**2
        E =  E1 + E2 + E3
        # E = np.sum([2 ** h * z for h, z in enumerate(z_h)]) + K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)]) + P * alpha_s ** 2
        expectation += E * prob

    return expectation, counts

In [10]:
from scipy.optimize import minimize
from qiskit_algorithms.optimizers import COBYLA


# 优化 QAOA 参数
def optimize_qaoa(n, m, l, K1, P, p, params):
    def objective(params):
        expectation, _ = cost_function(params, n, m, l, K1, P, p)

        return expectation

    # optimizer = Nelder-Mead(rhobeg = 1.5, tol = 1e-7)
    # params = np.random.rand(2 * p_max) * 2 * np.pi
    # result = optimizer.minimize(fun=objective, x0=params)
    result = minimize(fun=objective, x0=params, method='Nelder-Mead',
                  options={'xatol': 1e-7})
    return result

In [11]:
import time

# 运行优化
# p_max = 6  # 最大层数

startTime = time.time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 0
final_result = object()
for p1 in range(1, 5):
    min_energy = 10000
    for k in range(300):
        print('第：', p1, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point
        beta_p = np.random.uniform(0, np.pi)
        gamma_p = np.random.uniform(0,  2 * np.pi)
        if k >0:
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
        init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        # 创建参数向量
        # print("init_point_temp"+ init_point_temp)
        result = optimize_qaoa(n, m, l, K1, P, p, init_point_temp)

        # print(result.x)
        optimal_params = result.x

        # 计算最终的能量
        energy, counts_temp = cost_function(result.x, n, m, l, K1, P, p)
        # 寻找这20对点中最优的参数
        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
endTime = time.time()
# params = np.random.rand(2 * p_max,2*np.pi)
# result = optimize_qaoa(n, m, l, K1, P, p, params)

第： 1 层   第 0 个参数
第： 1 层   第 1 个参数
第： 1 层   第 2 个参数
第： 1 层   第 3 个参数
第： 1 层   第 4 个参数
第： 1 层   第 5 个参数
第： 1 层   第 6 个参数
第： 1 层   第 7 个参数
第： 1 层   第 8 个参数
第： 1 层   第 9 个参数
第： 1 层   第 10 个参数
第： 1 层   第 11 个参数
第： 1 层   第 12 个参数
第： 1 层   第 13 个参数
第： 1 层   第 14 个参数
第： 1 层   第 15 个参数
第： 1 层   第 16 个参数
第： 1 层   第 17 个参数
第： 1 层   第 18 个参数
第： 1 层   第 19 个参数
第： 1 层   第 20 个参数
第： 1 层   第 21 个参数
第： 1 层   第 22 个参数
第： 1 层   第 23 个参数
第： 1 层   第 24 个参数
第： 1 层   第 25 个参数
第： 1 层   第 26 个参数
第： 1 层   第 27 个参数
第： 1 层   第 28 个参数
第： 1 层   第 29 个参数
第： 1 层   第 30 个参数
第： 1 层   第 31 个参数
第： 1 层   第 32 个参数
第： 1 层   第 33 个参数
第： 1 层   第 34 个参数
第： 1 层   第 35 个参数
第： 1 层   第 36 个参数
第： 1 层   第 37 个参数
第： 1 层   第 38 个参数
第： 1 层   第 39 个参数
第： 1 层   第 40 个参数
第： 1 层   第 41 个参数
第： 1 层   第 42 个参数
第： 1 层   第 43 个参数
第： 1 层   第 44 个参数
第： 1 层   第 45 个参数
第： 1 层   第 46 个参数
第： 1 层   第 47 个参数
第： 1 层   第 48 个参数
第： 1 层   第 49 个参数
第： 1 层   第 50 个参数
第： 1 层   第 51 个参数
第： 1 层   第 52 个参数
第： 1 层   第 53 个参数
第： 1 层   第 54 个参数
第： 1 层   第 55 个参数
第：

In [12]:
# 输出结果
print("Optimal parameters:", final_result.x)
print("Optimal value:", final_result.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation, final_counts = cost_function(final_result.x, n, m, l, K1, P, p)
# sorted_dict = dict(sorted(final_counts.items(), key=lambda item: item[1], reverse=True))
sorted_dict = {item[0][::-1]: item[1] for item in sorted(final_counts.items(), key=lambda item: item[1], reverse=True)}
print("Final expectation value:", final_expectation)
print("Final measurement counts:", sorted_dict)

Optimal parameters: [0.81863979 2.0125983  1.41609001 0.29129581 4.18649848 4.39254432
 2.50619687 3.25530092]
Optimal value: 97.68710000000007
Final expectation value: 97.68710000000007
Final measurement counts: {'0000000': 5328, '1000000': 1264, '0000010': 1211, '0000001': 526, '1000010': 295, '0100000': 248, '1000001': 135, '0000100': 87, '0000110': 81, '1000100': 75, '1100100': 73, '0100010': 50, '0000011': 47, '0010000': 42, '0001001': 39, '1001000': 31, '0110000': 30, '0100001': 28, '1000011': 27, '1001001': 22, '0101100': 22, '0010010': 22, '1010000': 21, '1100010': 17, '1100000': 17, '1000110': 16, '0011000': 16, '0001000': 16, '0100110': 15, '1001010': 14, '0001011': 14, '0001100': 10, '1101110': 10, '1001011': 10, '1100110': 8, '0100100': 8, '0101110': 8, '0100101': 8, '0000101': 7, '0110110': 5, '0000111': 5, '0100011': 5, '1001100': 5, '0010001': 4, '0110100': 4, '0110010': 4, '0001010': 4, '1101100': 4, '0101111': 3, '1001101': 3, '0010100': 3, '1010001': 3, '1100101': 3, 

In [13]:
expectation = 0
bitstring = "101001"
z_h = np.array([int(bit) for bit in bitstring[(n*m):]])
x_ij = np.array([int(bit) for bit in bitstring[:(n*m)]]).reshape(n, m)

E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
E2 = K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)])
# alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
E3 = 0
for j in range(m):

    sum_xp = np.sum([x_ij[i][j] * p[i][j] for i in range(n)])

    sum_zh = np.sum([2**h * z_h[h] for h in range(l)])

    max_value = max(0, sum_xp - sum_zh)

    E3 += P * (max_value)**2
E =  E1 + E2 + E3
print(E1)
print(E2)
print(E3)

IndexError: index 0 is out of bounds for axis 0 with size 0